In [7]:
import time
from binance.client import Client
import psycopg2
from psycopg2 import sql
import pandas as pd
import threading

# Initialize the Binance API client
api_key = 'vijG9ARQVjevGky2bkTgFACUMmRJBVxVqwtzDIONOToNIw4YmTMxMgyy4PEVUhFH'
api_secret = 'f9BHEpkASNW9S0nd7xe1Iq55ccd0bFrwI8LL0mA0UlIgODhubsX8noG6TnWicvmh'
client = Client()

# List of coins to fetch data for
coins = ['sxp', 'chess', 'blz', 'joe', 'perl']

def get_db_connection():
    connection = psycopg2.connect(
        host='ahsanpost.postgres.database.azure.com',
        user='ahsan',
        password='name@123',
        dbname='postdb',
        sslmode='require'
    )
    return connection

# Create tables for each coin
def create_tables():
    connection = get_db_connection()
    cursor = connection.cursor()
    for coin in coins:
        cursor.execute(sql.SQL("""
            CREATE TABLE IF NOT EXISTS {} (
                id SERIAL PRIMARY KEY,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                price NUMERIC(18, 8),
                volume NUMERIC(18, 8)
            )
        """).format(sql.Identifier(f'{coin}usdt')))
    connection.commit()
    connection.close()

def fetch_and_store_recent_trades(coin):
    last_trade_timestamp = None
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            trades = client.get_recent_trades(symbol=f'{coin.upper()}USDT')
            for trade in trades:
                trade_timestamp = pd.to_datetime(trade['time'], unit='ms')
                if last_trade_timestamp is None or trade_timestamp > last_trade_timestamp:
                    cursor.execute(sql.SQL("INSERT INTO {} (timestamp, price, volume) VALUES (%s, %s, %s)").format(sql.Identifier(f'{coin}usdt')), (trade_timestamp, trade['price'], trade['qty']))
                    last_trade_timestamp = trade_timestamp
            connection.commit()
        except Exception as e:
            print(f'Error fetching and storing trades for {coin}: {e}')
            connection.rollback()
            # Wait for 60 seconds before retrying
            time.sleep(60)
            continue
        finally:
            connection.close()
        time.sleep(10)


# Function to delete data older than 1 hour
def delete_old_data(coin):
    while True:
        connection = get_db_connection()
        cursor = connection.cursor()
        try:
            cursor.execute(sql.SQL("DELETE FROM {} WHERE timestamp < NOW() - INTERVAL ' HOUR'").format(sql.Identifier(f'{coin}usdt')))
            connection.commit()
        except Exception as e:
            print(f'Error deleting old data for {coin}: {e}')
            connection.rollback()
        finally:
            connection.close()
        time.sleep(3000)  # Sleep for 5 minutes

# Start the threads
def start_threads():
    create_tables()
    for coin in coins:
        threading.Thread(target=fetch_and_store_recent_trades, args=(coin,)).start()
        threading.Thread(target=delete_old_data, args=(coin,)).start()

start_threads()

while True:
    time.sleep(2)


Error fetching and storing trades for joe: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for sxp: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for sxp: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for blz: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for perl: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for joe: ('Connection aborted.', ConnectionResetError(10054, 'An 

Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\3918153311.py", line 46, in fetch_and_store_recent_trades
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\2436743614.py", line 17, in get_db_connection
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\psycopg2\__init__.py", line 122, in connect
    conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
psycopg2.OperationalError: could not translate host name "ahsanpost.postgres.database.azure.com" to address: Unknown host


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\3918153311.py", 

Error fetching and storing trades for perl: could not translate host name "ahsanpost.postgres.database.azure.com" to address: Unknown host

Error fetching and storing trades for joe: could not translate host name "ahsanpost.postgres.database.azure.com" to address: Unknown host

Error fetching and storing trades for chess: could not translate host name "ahsanpost.postgres.database.azure.com" to address: Unknown host



Exception in thread Thread-27:
Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\2436743614.py", line 46, in fetch_and_store_recent_trades
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\2436743614.py", line 17, in get_db_connection
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\psycopg2\__init__.py", line 122, in connect
    conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
psycopg2.OperationalError: could not translate host name "ahsanpost.postgres.database.azure.com" to address: Unknown host


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\2436743614.py", 

Error fetching and storing trades for perl: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for perl: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for blz: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=10)


...

Error fetching and storing trades for blz: connection to server at "ahsanpost.postgres.database.azure.com" (20.163.232.246), port 5432 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



Exception in thread Thread-25:
Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\2436743614.py", line 46, in fetch_and_store_recent_trades
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\2436743614.py", line 17, in get_db_connection
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\psycopg2\__init__.py", line 122, in connect
    conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
psycopg2.OperationalError: connection to server at "ahsanpost.postgres.database.azure.com" (20.163.232.246), port 5432 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
SSL SYSCALL error: Connection reset by peer (0x00002746/10054)


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 932, in _bootstrap_inner
    self.

Error fetching and storing trades for sxp: connection to server at "ahsanpost.postgres.database.azure.com" (20.163.232.246), port 5432 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
SSL SYSCALL error: Connection reset by peer (0x00002746/10054)



Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\3918153311.py", line 46, in fetch_and_store_recent_trades
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_19556\2436743614.py", line 17, in get_db_connection
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\site-packages\psycopg2\__init__.py", line 122, in connect
    conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
psycopg2.OperationalError: connection to server at "ahsanpost.postgres.database.azure.com" (20.163.232.246), port 5432 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
SSL SYSCALL error: Connection reset by peer (0x00002746/10054)


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 932, in _bootstrap_inner
    self.r

Error fetching and storing trades for sxp: connection to server at "ahsanpost.postgres.database.azure.com" (20.163.232.246), port 5432 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
SSL SYSCALL error: Connection reset by peer (0x00002746/10054)

Error fetching and storing trades for chess: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for chess: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for chess: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching and storing trades for chess: ('Connection aborted.', ConnectionResetError(10054, 'An existing 